# baseline v3

이 베이스라인 코드는 `사전학습 모델 로드`, `배치 학습`, `파인튜닝`, `양자화`, `PEFT` 등이 적용된 버전입니다.

Colab의 GPU 환경에서 개발되었습니다.
- 런타임 - 런타임 유형 변경 - GPU로 변경(T4 GPU 등)



# 환경 준비

개발 환경에 필요한 라이브러리 버전을 고정하고 최신 버전으로 라이브러리를 업데이트합니다.

- 아래 셀 실행
- 실행 완료 후 런타임 - 세션 다시 시작

In [ ]:
!pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pillow pandas torch torchvision --upgrade

# 데이터 준비

개발에 필요한 데이터를 준비합니다.

- train.csv, train 폴더
- test.csv, test 폴더
- sample_submission.csv

본 베이스라인은 colab에서 구글 드라이브를 마운트하여 사용합니다.

데이터를 압축 해제하는데 몇 분 정도의 시간이 소요됩니다.

#### 실습 참고 내용

    챕터 2-2 합성 데이터 실습
    - 구글 드라이브 마운트 : drive()

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 압축 해제
!unzip "/content/drive/My Drive/data.zip" -d "/content/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/test/test_2776.jpg  
  inflating: /content/test/test_2777.jpg  
  inflating: /content/test/test_2778.jpg  
  inflating: /content/test/test_2779.jpg  
  inflating: /content/test/test_2780.jpg  
  inflating: /content/test/test_2781.jpg  
  inflating: /content/test/test_2782.jpg  
  inflating: /content/test/test_2783.jpg  
  inflating: /content/test/test_2784.jpg  
  inflating: /content/test/test_2785.jpg  
  inflating: /content/test/test_2786.jpg  
  inflating: /content/test/test_2787.jpg  
  inflating: /content/test/test_2788.jpg  
  inflating: /content/test/test_2789.jpg  
  inflating: /content/test/test_2790.jpg  
  inflating: /content/test/test_2791.jpg  
  inflating: /content/test/test_2792.jpg  
  inflating: /content/test/test_2793.jpg  
  inflating: /content/test/test_2794.jpg  
  inflating: /content/test/test_2795.jpg  
  inflating: /content/test/test_2796.jpg  
  inflating: /content/test/test_2797.jpg  
  inflating: /cont

In [ ]:
# Pillow 오류 방지용 버전 명시 포함
!pip install --upgrade --force-reinstall Pillow==10.4.0

# 라이브러리, 데이터, 설정

In [ ]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
MODEL_ID = "Qwen/Qwen3-VL-4B-Instruct"   # ✅ Qwen3-VL-4B 사용
IMAGE_SIZE = 448   # ✅ 해상도 값 448로 높임
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# dtype: bf16 지원 시 bf16, 아니면 fp16 (셀 2에서 모델 로드시 사용)
amp_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

# 데이터셋 로드
train_df = pd.read_csv("/content/train.csv")
test_df  = pd.read_csv("/content/test.csv")

# ✅학습데이터 400개 
train_df = train_df.sample(n=400, random_state=SEED).reset_index(drop=True)

print(f"train_df: {len(train_df)}, test_df: {len(test_df)}")


Device: cuda
train_df: 400, test_df: 3887


In [ ]:
# === Utils: 프롬프트/선지 파싱/TTA ===
import torch, os
from collections import Counter
from PIL import Image

def extract_choice(text: str) -> str:
    text = (text or "").strip().upper()
    # 끝 글자 우선
    if text and text[-1] in "ABCD":
        return text[-1]
    # 마지막 라인 스캔
    lines = [l.strip().upper() for l in text.splitlines() if l.strip()]
    if lines:
        last = lines[-1]
        for tok in ("A","B","C","D"):
            if tok in last: return tok
    # 전체 스캔
    for tok in ("A","B","C","D"):
        if tok in text: return tok
    return "A"  # fallback

def build_mc_prompt(q, a, b, c, d):
    return (
        "[문제]\n"
        f"{q}\n\n"
        "[보기]\n"
        f"A) {a}\nB) {b}\nC) {c}\nD) {d}\n\n"
        "형식: 한 글자만 답하라.\n정답:"
    )

@torch.no_grad()
def generate_choice(model, processor, image, prompt, max_new_tokens=4):
    model.eval()
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
    out_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        repetition_penalty=1.0
    )
    out = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    tail = out[-16:]
    return extract_choice(tail)

def tta_predict_choices(model, processor, image, prompt):
    # 좌우반전 한 번 TTA (다수결)
    from PIL import ImageOps
    p1 = generate_choice(model, processor, image, prompt)
    p2 = generate_choice(model, processor, ImageOps.mirror(image), prompt)
    return Counter([p1, p2]).most_common(1)[0][0]


# 모델, Processor

7.5GB 정도의 모델 다운로드가 진행됩니다. 10~20분 정도가 소요됩니다.

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - LoRA 구현 : LoraConfig()

In [ ]:
# === EMA(지수이동평균) ===
class EMAHelper:
    def __init__(self, model, decay=0.999):
        self.decay = decay
        self.shadow = {}
        for n, p in model.named_parameters():
            if p.requires_grad:
                self.shadow[n] = p.detach().clone()

    @torch.no_grad()
    def update(self, model):
        for n, p in model.named_parameters():
            if p.requires_grad:
                self.shadow[n].mul_(self.decay).add_(p.detach(), alpha=1.0 - self.decay)

    @torch.no_grad()
    def swap_to_ema(self, model):
        # 현재 ↔ EMA 가중치 스왑 (두 번 호출하면 원복)
        for n, p in model.named_parameters():
            if p.requires_grad:
                tmp = p.detach().clone()
                p.data.copy_(self.shadow[n].data)
                self.shadow[n] = tmp

# === Validation: 객관식 정확도 ===
@torch.no_grad()
def evaluate_mc_accuracy(model, processor, valid_df, use_tta=False, progress=False):
    model.eval()
    N = len(valid_df)
    correct = 0
    it = range(N)
    if progress:
        try:
            from tqdm.auto import tqdm
            it = tqdm(it, total=N, desc="Validate", unit="sample")
        except Exception:
            pass

    for i in it:
        row = valid_df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")
        prompt = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])
        pred = tta_predict_choices(model, processor, img, prompt) if use_tta else generate_choice(model, processor, img, prompt)
        gt = str(row["answer"]).strip().upper()  # ← 정답 컬럼명 다르면 여기만 수정
        correct += (pred == gt)
    return correct / max(1, N)


In [ ]:
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 양자화: 4bit NF4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=amp_dtype,   # 셀1에서 정의한 amp_dtype 사용
)

# 프로세서 (이미지 해상도 고정: 448x448)
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE * IMAGE_SIZE,
    max_pixels=IMAGE_SIZE * IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델 (4bit 로드 + dtype 지정 + device map)
base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=amp_dtype,
    device_map="auto",
    trust_remote_code=True,
)

# 체크포인트/메모리 안정화
base_model.config.use_cache = False
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

# LoRA 세팅 ✅
# - 학습데이터 400개에 epoch 2를 기준으로 LoRA 세팅 값을 조절
# LoRA 세팅 ✅
# - 학습데이터 400개, epoch 2 기준으로 최적화된 설정
# - 목표: 메모리 절약(T4 12GB) + 안정적인 수렴 + 과적합 방지

lora_config = LoraConfig(
    r=16,                               # 🔹 랭크(rank)
    # LoRA의 저랭크 행렬 차원 크기. r이 클수록 표현력 ↑, 메모리/시간 부담 ↑
    # 일반적으로 4~16 사이에서 조정. r=16은 중간 이상 수준으로 성능 향상 기대 가능.

    lora_alpha=32,                      # 🔹 LoRA 스케일링 계수
    # LoRA의 학습된 가중치(ΔW)를 얼마나 크게 반영할지 결정.
    # α/r 비율이 학습 효과를 조절함 → 여기선 32/16=2로 안정적이며 과적합 위험 낮음.

    lora_dropout=0.05,                  # 🔹 드롭아웃 비율
    # LoRA 가중치 업데이트 시 일부를 랜덤 무시(regularization).
    # 0.05는 적당히 낮은 확률로, 과적합 방지 + 미세조정 안정성 향상.

    bias="none",                        # 🔹 bias 학습 여부
    # 기존 모델의 bias는 그대로 두고, LoRA 레이어만 학습.
    # 파라미터 수를 줄여 메모리 절약 및 학습 안정성 확보.

    target_modules=[                    # 🔹 적용 대상 모듈
        "q_proj","k_proj","v_proj",     # → Attention의 Query, Key, Value projection
        "o_proj",                       # → Attention Output projection
        "gate_proj","up_proj","down_proj" # → FFN(피드포워드 네트워크) 부분
    ],
    # 핵심 연산 모듈(QKV + FFN)에만 LoRA 적용 → 효율적 미세조정
    # 전체 파라미터 중 0.1~1%만 학습하면서도 원본 모델 성능 대부분 유지 가능.

    task_type="CAUSAL_LM",              # 🔹 작업 유형 'Causal Language Model'용 설정.
)


# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

print("✅ Qwen3-VL-4B 4bit + LoRA 준비 완료")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

trainable params: 33,030,144 || all params: 4,470,845,952 || trainable%: 0.7388
✅ Qwen3-VL-4B 4bit + LoRA 준비 완료


# 프롬프트 템플릿

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - 프롬프트 템플릿 : convert_to_chatml(), formatting_prompts_func()

In [ ]:
# ✅ 학습률을 높이기 위해 지시사항과 프롬프트 영문으로 바꿈

# 모델 지시사항
SYSTEM_INSTRUCT = (
    "You are a smart and accurate visual question answering assistant. "
    "Carefully look at the image and understand the question before answering. "
    "Answer using exactly one letter among a, b, c, or d, without any explanation."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "Answer only with one lowercase letter among a, b, c, or d. Do not include any other words."
    )

# Custom Dataset, Collator

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - TensorDataset()

    챕터 5-2 데이터 생성 및 파인튜닝 (향후 학습 분량)
    - IntentDataset()

In [ ]:
# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"])
        user_text = build_mc_prompt(q, a, b, c, d)

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]}
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role":"assistant","content":[{"type":"text","text":gold}]})

        return {"messages": messages, "image": img}

# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True

    def __call__(self, batch):
        texts, images = [], []
        for sample in batch:
            messages = sample["messages"]
            img = sample["image"]

            text = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
            images.append(img)

        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )

        if self.train:
            enc["labels"] = enc["input_ids"].clone()

        return enc


# DataLoader

#### 실습 참고 내용

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 데이터로더 정의 : DataLoader()

In [ ]:
# ===== DataLoader & Optimizer/Scheduler (중복 제거 버전) =====
# NOTE: DataLoader 임포트/amp_dtype 정의는 앞 셀에서 이미 했으니 여기서는 생략

# 검증용 데이터 분리
split = int(len(train_df) * 0.9)
train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# ===== Fast LoRA config (T4 12GB, ~20분 내) =====
# 🔹 목표: 제한된 GPU 메모리(12GB)에서 빠르고 안정적인 LoRA 학습 수행
# 🔹 특징: 적은 데이터셋(약 400개) + 2 epoch 기준으로 효율적으로 수렴하도록 설정

BATCH_SIZE   = 1            # ✅ DataLoader에서 한 번에 불러올 샘플 개수
# VRAM 절약용 최소 배치. 메모리 사용량 ↓ 대신, 아래 GRAD_ACCUM으로 유효 배치 크기를 키워 안정성 보완.
GRAD_ACCUM   = 8            # ✅ Gradient Accumulation Steps
# 그래디언트를 8번 누적한 뒤 한 번 optimizer step 수행.
# 유효 배치 크기 = 1(batch) × 8(accum) = 8 → 메모리는 적게 쓰면서 큰 배치 효과(더 부드러운 학습)를 얻음.
EPOCHS       = 2            # ✅ 전 400개의 데이터 기준으로 과적합 위험을 낮추면서 충분히 수렴.
MAX_STEPS    = 240          # ✅ # EPOCHS보다 우선 적용 → 240 step 도달 시 학습 종료.
LR           = 2e-4         # ✅ 학습률 (Learning Rate) 2e-4는 속도와 안정성 사이의 균형값 → 빠르게 수렴하면서 폭주 위험 낮음.
WEIGHT_DECAY = 0.01         # ✅ L2 정규화 계수
# 가중치가 과도하게 커지는 걸 방지 (overfitting 억제). 일반적인 Transformer의 표준값 (0.01).
WARMUP_STEPS = 20           # ✅ Learning Rate Warmup 단계 수
# 초반 20 step 동안 학습률을 선형 증가시켜 모델 예열. 갑작스런 큰 학습률로 인한 불안정한 손실 폭주 방지.
MAX_NORM     = 1.0          # ✅ Gradient Clipping 최대 노름 값
# 그래디언트 폭주 방지 (Exploding Gradient). norm이 1.0을 넘으면 잘라내서 안정적 업데이트 보장.
SEED         = 42           # ✅ 랜덤 시드 고정 (재현성)
# 데이터 셔플, weight 초기화 등의 무작위성을 통제. 실험 결과를 반복해도 동일하게 재현 가능.


# 성능 옵션
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=DataCollator(processor, True),
    pin_memory=True,  # CUDA일 때만 True로 바꾸고 싶으면: (device=="cuda")
)

valid_loader = DataLoader(
    valid_ds,
    batch_size=BATCH_SIZE * 2,  # 검증은 grad 없음 → 배치 2배 가능
    shuffle=False,
    collate_fn=DataCollator(processor, True),
    pin_memory=True,
)

# ----- Optimizer / Scheduler -----
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=MAX_STEPS // max(1, GRAD_ACCUM) + WARMUP_STEPS,
)

print(f"[Info] train_batches={len(train_loader)}, valid_batches={len(valid_loader)}")


[Info] train_batches=360, valid_batches=20


# fine-tuning

- 200개만 학습 : 10~20분 소요

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - 모델 정의 : SimpleMLP(), SequentialMLP()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
# ===== Fine-tuning Loop (중복 제거 버전) =====
import math, torch
from tqdm.auto import tqdm

# GradScaler: fp16일 때만 활성화 (bf16이면 비활성)
use_fp16_scaler = (torch.cuda.is_available() and amp_dtype == torch.float16)
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16_scaler)

# 모델은 이미 device_map="auto"로 로드됨 → model.to(...) 호출하지 않음
model.train()
global_step = 0

print(f"Starting training with amp_dtype={amp_dtype}, "
      f"epochs={EPOCHS}, max_steps={MAX_STEPS}, grad_accum={GRAD_ACCUM}")

for epoch in range(1, EPOCHS + 1):
    running_loss = 0.0
    progress = tqdm(train_loader, desc=f"Epoch {epoch} [train]", unit="batch")

    optimizer.zero_grad(set_to_none=True)
    for step, batch in enumerate(progress, start=1):
        if global_step >= MAX_STEPS:
            break

        # 텐서만 모델의 디바이스로 이동
        batch = {k: (v.to(model.device, non_blocking=True) if isinstance(v, torch.Tensor) else v)
                 for k, v in batch.items()}

        # AMP 전방/역전파
        with torch.autocast(device_type="cuda", dtype=amp_dtype, enabled=torch.cuda.is_available()):
            outputs = model(**batch)
            loss = outputs.loss
            if GRAD_ACCUM > 1:
                loss = loss / GRAD_ACCUM

        if use_fp16_scaler:
            scaler.scale(loss).backward()
        else:
            loss.backward()

        # grad accumulation
        if step % GRAD_ACCUM == 0:
            # gradient clipping
            if use_fp16_scaler:
                scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_NORM)

            # optimizer / scheduler step
            if use_fp16_scaler:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()

            global_step += 1
            progress.set_postfix({"loss": f"{(loss.item() * GRAD_ACCUM):.4f}", "gs": global_step})

        running_loss += float(loss)

    # 조기 종료 조건
    if global_step >= MAX_STEPS:
        print(f"⏹️ Reached MAX_STEPS={MAX_STEPS}. Stopping.")
        break

    # ===== Validation =====
    model.eval()
    val_loss, val_steps = 0.0, 0
    with torch.no_grad():
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch} [valid]", unit="batch"):
            vb = {k: (v.to(model.device, non_blocking=True) if isinstance(v, torch.Tensor) else v)
                  for k, v in vb.items()}
            with torch.autocast(device_type="cuda", dtype=amp_dtype, enabled=torch.cuda.is_available()):
                out = model(**vb)
                vloss = out.loss if hasattr(out, "loss") and out.loss is not None else 0.0
            val_loss += float(vloss)
            val_steps += 1

    mean_vloss = val_loss / max(val_steps, 1)
    print(f"[Epoch {epoch}] valid loss: {mean_vloss:.4f}")
    model.train()

# ===== 저장 (LoRA 어댑터 + processor) =====
SAVE_DIR = "/content/qwen3_vl_4b_lora"
model.save_pretrained(SAVE_DIR)      
processor.save_pretrained(SAVE_DIR)
print("✅ Saved to:", SAVE_DIR)


Starting training with amp_dtype=torch.bfloat16, epochs=2, max_steps=240, grad_accum=8


/tmp/ipython-input-3465423891.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_fp16_scaler)


Epoch 1 [train]:   0%|          | 0/360 [00:00<?, ?batch/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/tmp/ipython-input-3465423891.py:60: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  running_loss += float(loss)


Epoch 1 [valid]:   0%|          | 0/20 [00:00<?, ?batch/s]

[Epoch 1] valid loss: 4.8559


Epoch 2 [train]:   0%|          | 0/360 [00:00<?, ?batch/s]

Epoch 2 [valid]:   0%|          | 0/20 [00:00<?, ?batch/s]

[Epoch 2] valid loss: 4.9018
✅ Saved to: /content/qwen3_vl_4b_lora


# inference

30분~1시간 소요

#### 실습 참고 내용

    챕터4-1 RAG 기반 Customer Service AI 에이전트 개발
    - 데이터 파서 : langchain_core.output_parsers(), StrOutputParser()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
# 데이터 파서 : 모델의 응답에서 선지를 추출
def extract_choice(text: str) -> str:
    text = text.strip().lower()

    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a"
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last

    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a"

# 추론을 위해 모든 레이어 활성화
model.eval()
preds = []

# 추론 루프
for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[
            {"type":"image","image":img},
            {"type":"text","text":user_text}
        ]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
        out_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False,
                                 eos_token_id=processor.tokenizer.eos_token_id)
    output_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    # print("output_text:", output_text)
    # print("extract_choice:", extract_choice(output_text))
    preds.append(extract_choice(output_text))

# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("/content/submission.csv", index=False)
print("Saved /content/submission.csv")

Inference:   0%|          | 0/3887 [00:00<?, ?sample/s]

Saved /content/submission.csv


In [ ]:
# 제출 파일 생성
save_path1 = "/content/drive/My Drive/epoch2_submission.csv"
submission.to_csv(save_path1, index=False)
print(f"Saved: {save_path1}")

Saved: /content/drive/My Drive/epoch2_submission.csv
